<a href="https://colab.research.google.com/github/atick-faisal/Sign-Language-Recognition/blob/main/core/colab/ProjectionNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown "1XG0u9AUABi_nKrWs5D4LGfP7v1KUFdD6"
!tar -xvf "lmc_dataset.tar.xz"

In [ ]:
import os
import joblib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models


# Configuration

In [ ]:
class config(object):
    INFERENCE_TIME = 3.0  # ... Segment duration
    HOLD_TIME = 2.0  # ... Time before next prediction session
    SEGMENT_LEN = 256
    IMG_LEN = 160  # 1/9 of the 224x224 image
    N_CHANNELS = 3
    LEARNING_RATE = 0.001
    BATCH_SIZE = 32
    N_EPOCHS = 100
    PATIENCE = 10
    PROJECTION_LANDMARKS = ["rp", "rf0", "rf1"]
    INFERENCE_FEATURES = [
        "rpx",
        "rpy",
        "rpz",
        "rf0x",
        "rf0y",
        "rf0z",
        "rf1x",
        "rf1y",
        "rf1z"
    ]
    MODEL_PREDICTION = "dev.atick.slr.model.prediction"
    FRAME_EVENT = "dev.atick.slr.frame"
    GESTURES = ["Good", "Bad", "Fine", "Hello", "Yes", "Deaf", "Me",
                "No", "Please", "Sorry", "Thank You", "You", "Hungry", "Goodbye"]
    FEATURE_NAMES = [
        "time",
        "rpx",
        "rpy",
        "rpz",
        "lpx",
        "lpy",
        "lpz",
        "rf0x",
        "rf0y",
        "rf0z",
        "rf1x",
        "rf1y",
        "rf1z",
        "rf2x",
        "rf2y",
        "rf2z",
        "rf3x",
        "rf3y",
        "rf3z",
        "rf4x",
        "rf4y",
        "rf4z",
        "lf0x",
        "lf0y",
        "lf0z",
        "lf1x",
        "lf1y",
        "lf1z",
        "lf2x",
        "lf2y",
        "lf2z",
        "lf3x",
        "lf3y",
        "lf3z",
        "lf4x",
        "lf4y",
        "lf4z",
    ]


# Load Dataset

In [ ]:
X_train = joblib.load("X_train.joblib")
y_train = joblib.load("y_train.joblib")
X_test = joblib.load("X_test.joblib")
y_test = joblib.load("y_test.joblib")

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

# Model Construction

In [ ]:
class ProjectionNet():
    def __init__(
        self,
        n_classes: int,
        base_model: tf.keras.Model,
        feature_vector_shape: tuple = (150, 1),
        projection_img_shape: tuple = (160, 160, 3),
        n_feature_vectors: int = 6,
        n_projections: int = 3
    ):
        self.projection_img_shape = projection_img_shape
        self.feature_vector_shape = feature_vector_shape
        self.n_feature_vectors = n_feature_vectors
        self.n_projections = n_projections
        self.base_model = base_model
        self.base_model.trainable = False

        self.dropout = layers.Dropout(0.5)

        self.mlp1 = layers.Dense(
            units=128,
            activation="relu"
        )

        self.mlp2 = layers.Dense(
            units=n_classes,
            activation="softmax"
        )

    def conv_block_1d(self):
        inputs = layers.Input(shape=self.feature_vector_shape)
        x = layers.BatchNormalization()(inputs)
        x = layers.Conv1D(8, 3, activation="selu")(x)
        x = layers.Conv1D(8, 3, activation="selu")(x)
        x = layers.MaxPool1D(2)(x)
        x = layers.Conv1D(16, 3, activation="selu")(x)
        x = layers.Conv1D(16, 3, activation="selu")(x)
        x = layers.MaxPool1D(2)(x)
        x = layers.Flatten()(x)
        output = layers.Dense(32)(x)

        return inputs, output

    def conv_block_2d(self):
        preprocess = layers.experimental.preprocessing.Rescaling(
            scale=1.0/127.5,
            offset=-1
        )
        inputs = layers.Input(shape=self.projection_img_shape)
        x = preprocess(inputs)
        x = self.base_model(x, training=False)
        output = layers.GlobalAveragePooling2D()(x)

        return inputs, output

    def model(self):

        inputs = []
        features = []

        for _ in range(self.n_feature_vectors):
            input_1d, features_1d = self.conv_block_1d()
            inputs.append(input_1d)
            features.append(features_1d)

        for _ in range(self.n_projections):
            input_2d, features_2d = self.conv_block_2d()
            inputs.append(input_2d)
            features.append(features_2d)

        x = layers.concatenate(features, axis=-1)
        x = self.dropout(x)
        x = self.mlp1(x)
        x = self.dropout(x)
        output = self.mlp2(x)

        return models.Model(inputs, output)

# Training

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(config.IMG_LEN, config.IMG_LEN, config.N_CHANNELS),
    include_top=False,
    weights="imagenet"
)

model = ProjectionNet(
    n_classes=len(config.GESTURES),
    base_model=base_model
).model()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=config.PATIENCE,
        restore_best_weights=True
    )
]

history = model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test, y_test),
    batch_size=config.BATCH_SIZE,
    epochs=config.N_EPOCHS,
    verbose=1,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams.update({
    # "text.usetex": True,
    "font.family": "serif",
    # "font.serif": ["Computer Modern Roman"],
    "font.size": 22,
    "text.color": "#212121",
    "axes.edgecolor": "#212121",
    "xtick.color": "#212121",
    "ytick.color": "#212121",
    "axes.labelcolor": "#212121",
    'legend.frameon': False,
})

fig = plt.figure(figsize=(8, 6))
ax = fig.gca()
ax.plot(history.history["loss"], "-", color="#212121", label="Train Loss")
ax.plot(history.history["val_loss"], "--",
        color="#212121", label="Validation Loss")
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.title("Learning Curves")
plt.legend()
plt.tight_layout()
# plt.savefig("../assets/lc.png")
plt.show()


# Evaluation

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
from sklearn.metrics import classification_report

y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred))

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

plt.figure(figsize=(12, 10))
cm = confusion_matrix(y_test, y_pred, normalize="true")
sns.heatmap(cm, cmap="summer")
plt.xticks(np.arange(len(config.GESTURES)) + 0.5, config.GESTURES, rotation=90)
plt.yticks(np.arange(len(config.GESTURES)) + 0.5, config.GESTURES, rotation=0)
plt.tight_layout()
plt.savefig("cm.svg")